In [1]:
import getpass
import json
import uuid

from kgforge.core import KnowledgeGraphForge

from inference_tools.utils import (check_premises,
                                   execute_query,
                                   execute_query_pipe,
                                   apply_rule,
                                   get_rule_parameters)

In [4]:
LOCAL_TOKEN = True 


def allocate_forge_session(org, project):
    config_path = "./config.yaml"
    ENDPOINT = "https://bbp.epfl.ch/nexus/v1"  

    if LOCAL_TOKEN: 
        with open('token.txt') as f:
            TOKEN = f.read()
    else: 
        TOKEN = getpass.getpass()
        
    searchendpoints = {
        "sparql": {
           "endpoint": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
       }
    }
    
    return KnowledgeGraphForge(
        config_path,
        endpoint=ENDPOINT,
        token=TOKEN, 
        bucket=f"{org}/{project}", 
        searchendpoints=searchendpoints,
        debug=True
    )

# [Create a Rule able to Generalize data across the brain region hierarchy](https://bbpteam.epfl.ch/project/issues/browse/DKE-1105)

In [5]:
rule_id = f"https://bbp.epfl.ch/neurosciencegraph/data/{uuid.uuid4()}"

searchQuery = {
    "type": "QueryPipe",

    "head": {
        "type": "SparqlQuery",
        "hasBody": 
        """
            SELECT ?id ?br
            WHERE { 
                ?id rdf:type $TypeQueryParameter .
                ?id nsg:brainLocation/nsg:brainRegion ?br .
                }
        """,
        "hasParameter": [
            {
                "type": "uri",
                "description": "paths to the properties being checked",
                "name": "TypeQueryParameter"
            },
        ],
        "queryConfiguration": {
            "sparlqlView": {
                "id": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
            },
            "org": "bbp",
            "project": "atlas"
        },
        "resultParameterMapping": [
            {
                "parameterName": "entityBr",
                "path": "br"
            },
            {
                "parameterName": "entity",
                "path": "id"
            }
        ]
    },
    "rest": {
        "type": "SparqlQuery",
        "hasBody": """

            SELECT ?id
            WHERE {
                ?br rdfs:subClassOf BrainRegion ;
                <http://schema.org/isPartOf>* $BrainRegionQueryParameter.
                VALUES (?id) {
                    $entity
                }.
                VALUES (?br){
                    $entityBr
                }
            }
            
        """,
        "hasParameter": [
        {
            "type": "sparql_value_list",
            "description": "List of URIs of brains regions",
            "name": "entityBr"
        }, 
        {
            "type": "sparql_value_list",
            "description": "List of URIs of entities of type",
            "name": "entity"
        }, 
        {
            "type": "uri",
            "description": "URI of the query brain region",
            "name": "BrainRegionQueryParameter"
        }
        ],
        "queryConfiguration": {
            "sparlqlView": {
                "id": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
            },
            "org": "neurosciencegraph",
            "project": "datamodels"
        },
        "resultParameterMapping": {
            "parameterName": "test",
            "path": "entity"
        }
    }
}

my_rule = {
    "id": rule_id,
    "type": "DataGeneralizationRule",
    "_schemaProject": "https://bbp.epfl.ch/nexus/v1/projects/bbp/inference-rules",
    "description": "Generalise up (ancestors) and/or down (descendants) a (combination of)"+
    "hierarchy in a BBP ontology (e.g. cell type, brain region)",
    "name": "TODO", #TODO
    "searchQuery": searchQuery,
}

input_filters = {
    "BrainRegionQueryParameter": "<http://api.brain-map.org/api/v2/data/Structure/315>",
    "TypeQueryParameter": "<https://neuroshapes.org/NeuronMorphology>"
}

res = apply_rule(allocate_forge_session, my_rule, input_filters)

QueryingError: 504 Server Error: Gateway Time-out for url: https://bbp.epfl.ch/nexus/v1/views/neurosciencegraph/datamodels/https%3A%2F%2Fbbp.epfl.ch%2Fneurosciencegraph%2Fdata%2Fviews%2Faggreg-sp%2Fdataset/sparql

# Other

In [ ]:
bbp_atlas_forge = allocate_forge_session("bbp", "atlas")

In [68]:
iso_id = "http://api.brain-map.org/api/v2/data/Structure/315"
# print(bbp_atlas_forge.retrieve(iso_id, cross_bucket=True))

In [6]:
nm = bbp_atlas_forge.search({"type": "NeuronMorphology"}, cross_bucket=True)

In [8]:
nm = bbp_atlas_forge.as_json(nm)

In [6]:
# nm[0]

In [37]:
[nm_i["identifier"] if "identifier" in nm_i else "idk" for nm_i in nm if "identifier"]

['AA0591',
 'AA0048',
 'AA0176',
 'AA0016',
 'AA0344',
 'AA0518',
 'AA0595',
 'AA0315',
 'AA0530',
 'AA0368',
 'AA0338',
 'AA0341',
 'AA0049',
 'AA0755',
 'AA0756',
 'AA0586',
 'AA0925',
 'AA0331',
 'AA0694',
 'AA0370',
 'AA0310',
 'AA0580',
 'AA0314',
 'AA0311',
 'AA0520',
 'AA0920',
 'AA0348',
 'AA0593',
 'AA0364',
 'AA0388',
 'AA0918',
 'AA0353',
 'AA0453',
 'AA0451',
 'AA0519',
 'AA0015',
 'AA0312',
 'AA0539',
 'AA0342',
 'AA0073',
 'AA0321',
 'AA0138',
 'AA0335',
 'AA0581',
 'AA0718',
 'AA0540',
 'AA0349',
 'AA0297',
 'AA0391',
 'AA0339',
 'AA0129',
 'AA0313',
 'AA0350',
 'AA0051',
 'AA0579',
 'AA0318',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',
 'idk',


In [ ]:
# filters = {
#     "type": "NeuronMorphology", 
#     "brainLocation" : {
#         "brainRegion": {
#             "id":iso_id
#         }
#     }
# }

# resources = bbp_atlas_forge.search(filters, cross_bucket=True)
# len(resources)

In [ ]:
### Check results are the same

In [68]:
test = """
    SELECT ?id ?br
    WHERE { 
        ?id ?path ?value .
        ?id nsg:brainLocation/nsg:brainRegion ?br .
        VALUES (?path ?value) {
            (rdf:type <https://neuroshapes.org/NeuronMorphology>)
            (contribution/agent <https://bbp.epfl.ch/neurosciencegraph/data/7c47aa15-9fc6-42ec-9871-d233c9c29028>)
        }
    }
"""

t = bbp_atlas_forge.sparql(test, rewrite=True, debug=True)
t = bbp_atlas_forge.as_json(t)
print(len(t))

test2 = """
    SELECT ?id ?br
    WHERE { 
       ?id rdf:type <https://neuroshapes.org/NeuronMorphology> .
       ?id contribution/agent <https://bbp.epfl.ch/neurosciencegraph/data/7c47aa15-9fc6-42ec-9871-d233c9c29028> .
       ?id nsg:brainLocation/nsg:brainRegion ?br .
   }
"""

#TODO is values an and or an or??? IT seems like an or which is not what we want
t2 = bbp_atlas_forge.sparql(test2, rewrite=True, debug=True)
t2 = bbp_atlas_forge.as_json(t2)
print(len(t2))

Submitted query:
   PREFIX bmc: <https://bbp.epfl.ch/ontologies/core/bmc/>
   PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
   PREFIX commonshapes: <https://neuroshapes.org/commons/>
   PREFIX datashapes: <https://neuroshapes.org/dash/>
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
   PREFIX oa: <http://www.w3.org/ns/oa#>
   PREFIX obo: <http://purl.obolibrary.org/obo/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFI

QueryingError: 400 Client Error: Bad Request for url: https://bbp.epfl.ch/nexus/v1/views/bbp/atlas/https%3A%2F%2Fbbp.epfl.ch%2Fneurosciencegraph%2Fdata%2Fviews%2Faggreg-sp%2Fdataset/sparql

In [8]:
query = """
SELECT ?id ?br
WHERE { 
    ?id type NeuronMorphology .
    ?id brainLocation/brainRegion ?br .
    }
"""

t = bbp_atlas_forge.sparql(query)
t = bbp_atlas_forge.as_json(t)
print(len(t))

brs = " ".join(["(<{}>)".format(thing["br"]) for thing in t])
ids = " ".join(["(<{}>)".format(thing["id"]) for thing in t])

query2 = """
            SELECT ?id
               WHERE {
                   ?br rdfs:subClassOf nsg:BrainRegion ;
                   <http://schema.org/isPartOf>* <http://api.brain-map.org/api/v2/data/Structure/315>.
                   VALUES (?id) { %s } .
                   VALUES (?br) { %s }
""" % (ids, brs)

# query2
# t2 = bbp_atlas_forge.sparql(query2, rewrite=True)

628


In [89]:
# query3 =  """
# SELECT ?id ?br
# WHERE { 
#     ?id brainLocation/brainRegion ?br .
#     ?br subClassOf BrainRegion .
#     }
# """

# t3 = bbp_atlas_forge.sparql(query3, rewrite=True, debug=False)
# t3 = [thing["br"] for thing in bbp_atlas_forge.as_json(t3)]
# len(t3)

In [90]:
# bbp_atlas_forge.search( {"hasPart": test}, cross_bucket=True, debug=True)

In [230]:
# query = """
# SELECT ?id
# WHERE { 
#     ?id brainLocation/brainRegion <http://api.brain-map.org/api/v2/data/Structure/315> .
#     }
# """
# t = bbp_atlas_forge.sparql(query, rewrite=True, debug=False)

# len(t)

In [94]:
# query = "SELECT ?id WHERE {?id schema:isPartOf* <http://api.brain-map.org/api/v2/data/Structure/315>;}" 
# t = bbp_atlas_forge.sparql(query, rewrite=True)
# test_br = list(set([thing["id"] for thing in bbp_atlas_forge.as_json(t)]))[:10]

In [91]:
# test_entity = bbp_atlas_forge.search({"type": "NeuronMorphology"}, cross_bucket=True)
# test_entity = [thing["id"] for thing in bbp_atlas_forge.as_json(test_entity)][:10]

In [25]:
res

[]

In [146]:
# rule_forge.register(rule_forge.from_json(my_rule))